In [ ]:
%matplotlib inline
import os
import sys
import gc
from tqdm.notebook import tqdm

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from commons.configuration_manager import ConfigurationManager
from src.learning.training.label_collector import LabelCollector
from src.learning.training.training_file_reader import TrainingFileReader
from src.learning.training.training_transformer import TrainingTransformer
from src.learning.models import create_cnn, create_mlp, create_multi_model

import pygame
import cv2

window_width = 500
window_height = 200

black = (0, 0, 0)
red = (255, 0, 0)
green = (0, 255, 0)
blue = (0, 0, 255)

pygame.init()
font = pygame.font.SysFont('Roboto', 20)

In [ ]:
config_manager = ConfigurationManager()
config = config_manager.config

reader = TrainingFileReader(path_to_training='../../training/laps/')
transformer = TrainingTransformer(config)
collector = LabelCollector()

path_to_training = '../../training/laps/'

def plot_stuff(title, plot_elems, figsize=(18, 10)):
    fig=plt.figure(figsize=figsize)
    plt.title(title)
    #plt.ylabel('dunno')
    plt.xlabel('Count')
    
    for plot_elem in plot_elems:
        plt.plot(plot_elem['data'], label=plot_elem['label'], alpha=plot_elem['alpha'])

    plt.grid(axis='y')
    plt.legend(loc='best')
    plt.show()
    
# axis=2 for frames, axis=0 for telem and diffs
def memory_creator(instance, memory, length=4, interval=2, axis=2):
    memory.append(instance)
    
    near_memory = memory[::-interval]
    if len(near_memory) < length:
        return None
    
    if len(memory) >= length * interval:
        memory.pop(0)
        
    return np.concatenate(near_memory, axis=axis)


def read_stored_data(filename):
    telemetry = reader.read_specific_telemetry_columns(filename + '.csv', collector.numeric_columns())
    diffs = reader.read_specific_telemetry_columns(filename + '.csv', collector.diff_columns())
    frames = reader.read_video(filename + '_resized.avi').astype(np.float32)
    transformer.normalize_video_for_training(frames)
    
    return frames, telemetry.to_numpy(), diffs.to_numpy()


def read_stored_video(filename):
    return reader.read_video(filename + '.avi')
    
    
def create_memorized_dataset(frames, telemetry, diffs, length, interval): 
    # final length diff is (length - 1) * interval
    mem_slice_frames = []
    mem_slice_telemetry = []
    
    len_diff = (length - 1) * interval
    mem_frames = np.zeros((frames.shape[0] - len_diff, *frames.shape[1:-1], frames.shape[-1] * length))
    mem_telems = np.zeros((telemetry.shape[0] - len_diff, telemetry.shape[1] * length))
    
    for i in range(0, frames.shape[0]):
        mem_frame = memory_creator(frames[i], mem_slice_frames, length=length, interval=interval, axis=2)
        mem_telem = memory_creator(telemetry[i], mem_slice_telemetry, length=length, interval=interval, axis=0)
        
        if mem_frame is not None:
            mem_frames[i - len_diff] = mem_frame
            mem_telems[i - len_diff] = mem_telem
            
    mem_diffs = diffs[len_diff:]
    
    assert mem_frames.shape[0] == mem_telems.shape[0] == mem_diffs.shape[0], "Lengths differ!"
    gc.collect()
    return mem_frames, mem_telems, mem_diffs
            
    
def skra_full_mem_datasets(filenames, length, interval):
    view_frames = []
    mem_frames = []
    mem_telemetry = []
    mem_diffs = []
    
    for training_file in filenames:
        frames_i = read_stored_video(training_file)
        mem_frames_i, mem_telemetry_i, mem_diffs_i = create_memorized_dataset(*read_stored_data(training_file), length, interval)
        
        mem_frames.append(mem_frames_i)
        mem_telemetry.append(mem_telemetry_i)
        mem_diffs.append(mem_diffs_i)
        view_frames.append(frames_i[((length - 1) * interval):])
            
        gc.collect()

    view_frames_np = np.concatenate(view_frames)
    mem_frames_np = np.concatenate(mem_frames)
    mem_telemetry_np = np.concatenate(mem_telemetry)
    mem_diffs_np = np.concatenate(mem_diffs)
    
    assert view_frames_np.shape[0] == mem_frames_np.shape[0], "Frames length mismatch!"
    assert mem_frames_np.shape[0] == mem_telemetry_np.shape[0] == mem_diffs_np.shape[0], "Mem lengths differ!"
    return mem_frames_np, mem_telemetry_np, mem_diffs_np, view_frames_np

In [ ]:
filename = 'lap_7_2020_01_24.csv'

df = pd.read_csv(path_to_training + filename)
#df = df[8000:25000]

print(df.shape)
print(df.columns.values)

steering = [{'data': df.d_steering, 'label':'expert diff', 'alpha':1.0}, 
         {'data': df.cs, 'label':'steering value', 'alpha':0.5}]

#plot_stuff('Steering', steering)

In [ ]:
filenames = ['lap_5_2020_01_24', 'lap_6_2020_01_24', 'lap_7_2020_01_24']
experiments = [(1, 1), (4, 1), (4, 3), (16, 1)]
epochs = 4
batch_size = 64
verbose = 1


# TODO if generator, then read 1 video for validation
mem_frames, mem_telems, mem_diffs, display_frames = skra_full_mem_datasets(filenames, *experiments[2])
frames_train, frames_test, telemetry_train, telemetry_test, diffs_train, diffs_test = train_test_split(mem_frames, mem_telems, mem_diffs, test_size=0.2)

mlp = create_mlp(input_shape=telemetry_train[0].shape)
cnn = create_cnn(input_shape=frames_train[0].shape)
multi = create_multi_model(mlp, cnn)

# TODO fitting with generator instead of in-memory data
multi.fit([telemetry_train, frames_train], diffs_train,
            validation_data=([telemetry_test, frames_test], diffs_test),
            epochs=epochs, batch_size=batch_size, verbose=verbose)

In [ ]:
pygame.display.init()
pygame.display.set_caption("Prediction viewer")
screen = pygame.display.set_mode((window_width, window_height))

try:
    for i in range(0, display_frames.shape[0]):
        frame = display_frames[i]
        screen.fill(black)
        
        converted_telem = mem_telems[i][np.newaxis, :]
        converted_frame = mem_frames[i].astype(np.float32)[np.newaxis, :]
        
        # gear, steering, throttle, braking
        predictions = multi.predict([converted_telem, converted_frame])[0]
        #print(mem_diffs[i])
        #print(predictions)
        
        steering = predictions[1]
        prediction_text = "{}".format(steering)
        if steering >= 0.0:
            prediction_texture = font.render(prediction_text, True, green)
        else:
            prediction_texture = font.render(prediction_text, True, blue)
        
        m_frame = mem_frames[i][:,:,:3]
        frame = np.rot90(frame)
        surface = pygame.surfarray.make_surface(frame)
        
        x = (window_width - frame.shape[0]) // 2
        y = 0
        
        screen.blit(prediction_texture, (200, window_height - 25))
        screen.blit(surface, (x, y))
        pygame.display.update()

        for event in pygame.event.get():
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE:
                    sys.exit(0)
    pygame.display.quit()
    
except (KeyboardInterrupt, SystemExit) as ex:
    pygame.display.quit()